<a href="https://colab.research.google.com/github/nicholasrichers/feature_selection-neuron/blob/main/Artigo_Feature_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
a = np.array([[1,2,2], [0,1,1], [3,2,1]])

In [15]:
a.dot(a.T)#*(1/14)

array([[ 9,  4,  9],
       [ 4,  2,  3],
       [ 9,  3, 14]])

In [12]:
np.cov(a)

array([[ 0.33333333,  0.33333333, -0.5       ],
       [ 0.33333333,  0.33333333, -0.5       ],
       [-0.5       , -0.5       ,  1.        ]])